In [1]:
import numpy as np
import scipy
from clock_project.simulation.magnitude_quantification import calculate_non_stationarity, calculate_ENS, random_nucleotide_distribution, calculate_information, entropy_calculation
from cogent3.maths.measure import jsd
from clock_project.simulation.wts import generate_rate_matrix
import scipy
import json
from clock_project.genome_analysis.yapeng_check_BV import get_bounds_violation, load_param_values
from cogent3.util.deserialise import deserialise_object
import numpy as np
import os
import glob
from cogent3 import get_app

import plotly.express as px

In [2]:
base_dir = '/Users/gulugulu/Desktop/honours/data_local/whole_genome_mammal87/triads_model_fitting_350_threshold'
gene_paths = glob.glob(os.path.join(base_dir, '*/'))
alignment_dir = '/Users/gulugulu/repos/PuningAnalysis/data/ensembl_ortholog_sequences/homologies_alignment_common_name_350_threshold'

In [86]:
def get_jsd_diff(triads_info):
    jsd_diff_dict = {}
    for identifier, info in triads_info.items():
        triads_info_value = info['triads_info_small_tree']
        triads_names = info['triads_species_names']
        jsd_dict = triads_info_value['jsd_dict']

        jsd_diff = abs(jsd_dict['JSD_difference'][triads_names['ingroup1']] - jsd_dict['JSD_difference'][triads_names['ingroup2']])
        jsd_diff_dict[identifier] = jsd_diff
    return jsd_diff_dict
        

In [4]:
def get_ingroup_jsd(triads_info):
    ingroup_jsd_dict = {}
    for identifier, info in triads_info.items():
        triads_info_value = info['triads_info_small_tree']
        triads_names = info['triads_species_names']
        jsd_dict = triads_info_value['jsd_dict']

        ingroup_jsd = jsd_dict['Ingroup_JSD']
        ingroup_jsd_dict[identifier] = ingroup_jsd
    return ingroup_jsd_dict

In [5]:
def get_ingroup_ens_diff(triads_info):
    ens_ingroup_dict = {}
    for identifier, info in triads_info.items():
        triads_info_value = info['triads_info_small_tree']
        triads_names = info['triads_species_names']
        ens_dict = triads_info_value['ens']
        ens_ingroup = np.log(ens_dict[triads_names['ingroup1']]/ ens_dict[triads_names['ingroup2']])
        ens_ingroup_dict[identifier] = ens_ingroup
    return ens_ingroup_dict

In [6]:
def get_ingroup_ens_absdiff(triads_info):
    ens_ingroup_dict = {}
    for identifier, info in triads_info.items():
        triads_info_value = info['triads_info_small_tree']
        triads_names = info['triads_species_names']
        ens_dict = triads_info_value['ens']
        ens_ingroup = abs(ens_dict[triads_names['ingroup1']]- ens_dict[triads_names['ingroup2']])
        ens_ingroup_dict[identifier] = ens_ingroup
    return ens_ingroup_dict

In [27]:
def get_nabla_absdiff(triads_info):
    nabla_diff_dict = {}
    for identifier, info in triads_info.items():
        triads_info_value = info['triads_info_small_tree']
        triads_names = info['triads_species_names']
        nabla_dict = triads_info_value['nabla_values']
        nbala_diff = abs(nabla_dict[triads_names['ingroup1']] - nabla_dict[triads_names['ingroup2']])
        nabla_diff_dict[identifier] = nbala_diff
    return nabla_diff_dict

In [7]:
def get_ingroup_nabla(triads_info):
    nabla_ingroup_dict = {}
    for identifier, info in triads_info.items():
        triads_info_value = info['triads_info_small_tree']
        triads_names = info['triads_species_names']
        nabla_dict = triads_info_value['nabla_values']
        nabla_difference = np.log(nabla_dict[triads_names['ingroup1']]/nabla_dict[triads_names['ingroup2']])
        nabla_ingroup_dict[identifier] = nabla_difference
    return nabla_ingroup_dict

In [8]:
# import scipy.stats

# nabla_log_ratio_dict = {}
# ens_log_ratio_dict = {}
# ens_diff_dict = {}
# nabla_diff_dict = {}
# nabla_average_dict = {}
# correlation_ens_nabla_log_ratio = []
# p_vlaue_ens_nabla_log_ratio = []
# correlation_ens_nabla_abs_diff = []
# p_vlaue_ens_nabla_abs_diff = []
# correlation_ens_diff_nabla_average = []
# p_vlaue_ens_diff_nabla_average = []
# correlation_ens_nabla = []
# p_vlaue_ens_nabla = []

# for path in gene_paths:
#     # Initializing lists to store computed values for each gene
#     nabla_log_ratio_list = []
#     ens_log_ratio_list = []
#     ens_diff_list = []
#     nabla_diff_list = []
#     nabla_average_list = []
#     nabla_list = []
#     ens_list = []

#     # Extract gene names from the path
#     gene_names = os.path.basename(path.rstrip('/'))

#     # Load the JSON data from the specified path
#     triads_data_path = os.path.join(path, 'triads_info_dict.json')
#     with open(triads_data_path, 'r') as file:
#         triads_info = json.load(file)

#     # Iterate through each entry in the triads data
#     for identifier, info in triads_info.items():
#         triads_info_value = info['triads_info_small_tree']
#         triads_names = info['triads_species_names']
#         nabla_dict = triads_info_value['nabla_values']
#         ens_dict = triads_info_value['ens']

#         # Calculate and store various metrics
#         nabla_log_ratio = np.log(nabla_dict[triads_names['ingroup1']] / nabla_dict[triads_names['ingroup2']])
#         nabla_log_ratio_list.append(nabla_log_ratio)
#         ens_log_ratio = np.log(ens_dict[triads_names['ingroup1']] / ens_dict[triads_names['ingroup2']])
#         ens_log_ratio_list.append(ens_log_ratio)
#         ens_diff = (ens_dict[triads_names['ingroup1']] -  ens_dict[triads_names['ingroup2']])**2
#         ens_diff_list.append(ens_diff)
#         nabla_diff = (nabla_dict[triads_names['ingroup1']] - nabla_dict[triads_names['ingroup2']])**2
#         nabla_diff_list.append(nabla_diff)
#         nabla_average = np.average([nabla_dict[triads_names['ingroup1']], nabla_dict[triads_names['ingroup2']]])
#         nabla_average_list.append(nabla_average)

#         nabla_list.extend(list(nabla_dict.values()))
#         ens_list.extend(list(ens_dict.values()))

#     #Add the correlation factor in the list
#     cor1, p_value1 = scipy.stats.spearmanr(ens_log_ratio_list, nabla_log_ratio_list)
#     correlation_ens_nabla_log_ratio.append(cor1)
#     p_vlaue_ens_nabla_log_ratio.append(p_value1)

#     cor2, p_value2 = scipy.stats.spearmanr(ens_diff_list, nabla_diff_list)
#     correlation_ens_nabla_abs_diff.append(cor2)
#     p_vlaue_ens_nabla_abs_diff.append(p_value2)
    
#     cor3, p_value3 = scipy.stats.spearmanr(ens_diff_list, nabla_average_list)
#     correlation_ens_diff_nabla_average.append(cor3)
#     p_vlaue_ens_diff_nabla_average.append(p_value3)
    
#     cor4, p_value4 = scipy.stats.spearmanr(ens_list, nabla_list)
#     correlation_ens_nabla.append(cor4)
#     p_vlaue_ens_nabla.append(p_value4)

#     # Store the lists into their respective dictionaries for the current gene
#     nabla_log_ratio_dict[gene_names] = nabla_log_ratio_list
#     ens_log_ratio_dict[gene_names] = ens_log_ratio_list
#     ens_diff_dict[gene_names] = ens_diff_list
#     nabla_diff_dict[gene_names] = nabla_diff_list
#     nabla_average_dict[gene_names] = nabla_average_list

In [9]:
from clock_project.maths.evolutionary_rate import calculate_stationary_distribution

def get_STI(triads_info, N):
    matrix_ingroup_dict = {}
    nuc_freq_ingroup_dict = {}
    STI_dict = {}
    chi_squared_dict = {}

    for identifier, info in triads_info.items():
        triads_info_value = info['triads_info_small_tree']
        triads_names = info['triads_species_names']
        matrix_dict = triads_info_value['matrices']
        nuc_freqs_dict = triads_info_value['nuc_freqs_dict']
        nuc_freqs_ingroup_pair = {
            triads_names['ingroup1']: nuc_freqs_dict['ingroup1'],
            triads_names['ingroup2']: nuc_freqs_dict['ingroup2']
        }
        nuc_freq_ingroup_dict[identifier] = nuc_freqs_ingroup_pair
        matrix_ingroup_pair = {
            triads_names['ingroup1']: matrix_dict[triads_names['ingroup1']],
            triads_names['ingroup2']: matrix_dict[triads_names['ingroup2']]
        }
        matrix_ingroup_dict[identifier] = matrix_ingroup_pair
        stationary_distribution_dict = {
            triads_names['ingroup1']: calculate_stationary_distribution(matrix_dict[triads_names['ingroup1']]),
            triads_names['ingroup2']: calculate_stationary_distribution(matrix_dict[triads_names['ingroup2']])
        }
        
        # Calculate STI values for each ingroup species
        for species, freqs in nuc_freqs_ingroup_pair.items():
            π = stationary_distribution_dict[species]
            p = freqs
            ΔC = p[1] - π[1]  # C
            ΔG = p[3] - π[3]  # G
            ΔA = p[2] - π[2]  # A
            ΔT = p[0] - π[0]  # T
            
            STI1 = ΔC + ΔG
            STI2 = ΔA - ΔT
            STI3 = ΔC - ΔG
            
            chi_squared = N * sum(((p[i] - π[i]) ** 2) / π[i] if π[i] > 0 else 0 for i in range(4))
            
            if identifier not in STI_dict:
                STI_dict[identifier] = {}
            if identifier not in chi_squared_dict:
                chi_squared_dict[identifier] = {}
                
            STI_dict[identifier][species] = (STI1, STI2, STI3)
            chi_squared_dict[identifier][species] = chi_squared

    return STI_dict, chi_squared_dict


In [10]:
alignment_length_dict = {}
alignment_dir_paths = glob.glob(os.path.join(alignment_dir, '*.json'))
for path in alignment_dir_paths:
    file_name = os.path.basename(path).rsplit('.', 1)[0]
    alignment = deserialise_object(json.load(open(path, 'r')))
    alignment_length_dict[file_name] = alignment.get_lengths()[0]


In [11]:
def process_path_STI(path):
    file_name = os.path.basename(path).rsplit('.', 1)[0]
    triads_data_path = os.path.join(path, 'triads_info_dict.json')
    triads_info = load_json_data(triads_data_path)
    alignment_length = alignment_length_dict[file_name]
    STI_dict, chi_squared_dict = get_STI(triads_info, alignment_length)
    return STI_dict, chi_squared_dict
    

In [13]:
def remove_outliers_iqr(data1, data2):
    def compute_iqr_bounds(data):
        Q1 = np.percentile(data, 25)
        Q3 = np.percentile(data, 75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        return lower_bound, upper_bound

    # Calculate IQR bounds for both lists
    lower_bound1, upper_bound1 = compute_iqr_bounds(data1)
    lower_bound2, upper_bound2 = compute_iqr_bounds(data2)

    # Filter out pairs where either value is an outlier
    filtered_data1 = []
    filtered_data2 = []

    for val1, val2 in zip(data1, data2):
        if (lower_bound1 <= val1 <= upper_bound1) and (lower_bound2 <= val2 <= upper_bound2):
            filtered_data1.append(val1)
            filtered_data2.append(val2)

    return filtered_data1, filtered_data2

In [108]:
import os
import json
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Function to load JSON data
def load_json_data(path):
    with open(path, 'r') as file:
        return json.load(file)

# Function to compute required values
def compute_values(path):
    nabla_log_ratio_list = []
    ens_log_ratio_list = []

    triads_data_path = os.path.join(path, 'triads_info_dict.json')
    triads_info = load_json_data(triads_data_path)
    nabla_log_ratio_dict = get_ingroup_nabla(triads_info)
    ens_log_ratio_dict = get_ingroup_ens_diff(triads_info)
    ens_abs_diff_dict = get_ingroup_ens_absdiff(triads_info)
    jsd_diff_dict = get_jsd_diff(triads_info)
    ingroup_jsd_dict = get_ingroup_jsd(triads_info)
    nabla_log_ratio_list = list(nabla_log_ratio_dict.values())
    nabla_absdiff_dict = get_nabla_absdiff(triads_info)
    ens_abs_diff_list = list(ens_abs_diff_dict.values())
    ens_log_ratio_list = list(ens_log_ratio_dict.values())
    jsd_diff_list = list(jsd_diff_dict.values())
    ingroup_jsd_list = list(ingroup_jsd_dict.values())
    nabla_absdiff_list = list(nabla_absdiff_dict.values())

    nabla_log_ratio_list_filtered, ens_log_ratio_list_filtered = remove_outliers_iqr(np.array(nabla_log_ratio_list), np.array(ens_log_ratio_list))
    ens_abs_diff_list_filtered, jsd_diff_list_filtered = remove_outliers_iqr(np.array(ens_abs_diff_list), np.array(jsd_diff_list))
    ens_abs_diff_list_filtered2, ingroup_jsd_list_filtered = remove_outliers_iqr(np.array(ens_abs_diff_list), np.array(ingroup_jsd_list))
    nabla_absdiff_filtered, ens_absdiff_filtered = remove_outliers_iqr(np.array(nabla_absdiff_list), np.array(ens_abs_diff_list))
    return nabla_log_ratio_list_filtered, ens_log_ratio_list_filtered, jsd_diff_list_filtered, ens_abs_diff_list_filtered, ingroup_jsd_list_filtered, ens_abs_diff_list_filtered2, nabla_absdiff_filtered, ens_absdiff_filtered

# Initialize the dictionary to store the data
gene_data_dict = {}

# Populate the dictionary with data for each gene
for path in gene_paths:
    gene_name = os.path.basename(path.rstrip('/'))
    nabla_log_ratio_list, ens_log_ratio_list, jsd_diff_list, ens_abs_diff_list, ingroup_jsd_list, ens_abs_diff_list2, nabla_absdiff, ens_absdiff = compute_values(path)
    gene_data_dict[gene_name] = {
        'nabla_log_ratio': nabla_log_ratio_list,
        'ens_log_ratio': ens_log_ratio_list,
        'ens_abs_diff': ens_abs_diff_list, 
        'jsd_diff': jsd_diff_list, 
        'ens_abs_diff2': ens_abs_diff_list2, 
        'ingroup_jsd': ingroup_jsd_list,
        'ens_abs_diff3': ens_absdiff,
        'nabla_absdiff': nabla_absdiff
    }






In [125]:

correlation_list1 = []
p_value_list1 = []
for gene, lists in gene_data_dict.items():
    nabla_log_ratio_list = lists['nabla_log_ratio']
    ens_log_ratio_list = lists['ens_log_ratio']
    #Add the correlation factor in the list
    cor, p_value = scipy.stats.pearsonr(list(ens_log_ratio_list), list(nabla_log_ratio_list))
    correlation_list1.append(cor)
    p_value_list1.append(p_value)

fig1 = px.box(correlation_list1, title= 'Pearson Correlation Test Nabla Log Ratio Vs. ENS Log Ratio')
# Update layout with labels and title
fig1.update_layout(
    template='plotly_white',
    margin=dict(l=20, r=20, t=50, b=20),
    autosize=True,
    yaxis_title_font={'size': 20},  
    xaxis_title_font={'size': 20}, 
    xaxis_title='Correlation Coefficient',
    yaxis_title='Value',
    width=None
)
fig1.show()
fig1.write_image('nabla_ens_cc.pdf')

In [127]:
gene_names = list(gene_data_dict.keys())
# Create the bar chart
fig = go.Figure(go.Bar(
    x=correlation_list1,
    y=gene_names,
    orientation='h',  # Horizontal bar chart
    marker=dict(color=correlation_list1, coloraxis="coloraxis")  # Color based on the correlation factor
))
fig.add_shape(
    type="line",
    x0=0.2, y0=0, x1=0.2, y1=len(gene_names),
    line=dict(color="yellow", width=3, dash="dashdot"),
)

fig.add_shape(
    type="line",
    x0=-0.2, y0=0, x1=-0.2, y1=len(gene_names),
    line=dict(color="yellow", width=3, dash="dashdot"),
)

# Update layout for better visualization
fig.update_layout(
    xaxis_title="Correlation Coefficient",
    yaxis_title="Gene ID",
    coloraxis=dict(colorscale='Viridis'),  # Red for higher, blue for lower values
    height=1200  # Adjust height based on the number of genes to avoid squeezing
)

fig.show()
fig.write_image('nabla_ens_cc_bar.pdf')

In [135]:
count = len([x for x in correlation_list1 if x < -0.2])
count

53

In [138]:
count = len([i for i in range(len(p_value_list1)) if p_value_list1[i] < 0.0003937 and correlation_list1[i] > 0.2])
count

9

In [121]:
correlation_list2 = []
p_value_list2 = []
for gene, lists in gene_data_dict.items():
    jsd_diff_list = lists['jsd_diff']
    ens_abs_diff_list = lists['ens_abs_diff']
    #Add the correlation factor in the list
    cor, p_value = scipy.stats.pearsonr(np.sqrt(jsd_diff_list), np.sqrt(ens_abs_diff_list))
    correlation_list2.append(cor)
    p_value_list2.append(p_value)

fig2 = px.box(correlation_list2, title= 'Pearson Correlation Test JSD Difference Vs. ENS difference')
# Update layout with labels and title
fig2.update_layout(
    template='plotly_white',
    margin=dict(l=20, r=20, t=50, b=20),
    autosize=True,
    yaxis_title_font={'size': 20},  
    xaxis_title_font={'size': 20}, 
    xaxis_title='Correlation Coefficient',
    yaxis_title='Value',
    width=None
)
fig2.show()
fig2.write_image('jsd_ens_diff_cc.pdf')

In [133]:
gene_names = list(gene_data_dict.keys())
# Create the bar chart
fig = go.Figure(go.Bar(
    x=correlation_list2,
    y=gene_names,
    orientation='h',  # Horizontal bar chart
    marker=dict(color=correlation_list1, coloraxis="coloraxis")  # Color based on the correlation factor
))
fig.add_shape(
    type="line",
    x0=0.2, y0=0, x1=0.2, y1=len(gene_names),
    line=dict(color="yellow", width=3, dash="dashdot"),
)


# Update layout for better visualization
fig.update_layout(
    xaxis_title="Correlation Coefficient",
    yaxis_title="Gene ID",
    coloraxis=dict(colorscale='Viridis'),  # Red for higher, blue for lower values
    height=1200  # Adjust height based on the number of genes to avoid squeezing
)

fig.show()
fig.write_image('jsd_ens_diff_cc_bar.pdf')

In [102]:
count = len([x for x in p_value_list2 if x < 0.0003937])
count

80

In [100]:
count = len([x for x in correlation_list2 if x > 0.2])
count

95

In [123]:
correlation_list3 = []
p_value_list3 = []
for gene, lists in gene_data_dict.items():
    ingroup_jsd_list = lists['ingroup_jsd']
    ens_abs_diff_list2 = lists['ens_abs_diff2']
    #Add the correlation factor in the list
    cor, p_value = scipy.stats.pearsonr(np.sqrt(ingroup_jsd_list), np.sqrt(ens_abs_diff_list2))
    correlation_list3.append(cor)
    p_value_list3.append(p_value)

fig3 = px.box(correlation_list3, title= 'Pearson Correlation Test Ingroup JSD Vs. ENS difference')
# Update layout with labels and title
fig3.update_layout(
    template='plotly_white',
    margin=dict(l=20, r=20, t=50, b=20),
    autosize=True,
    yaxis_title_font={'size': 20},  
    xaxis_title_font={'size': 20}, 
    xaxis_title='Correlation Coefficient',
    yaxis_title='Value',
    width=None
)
fig3.show()
fig3.write_image('ingroup_jsd_ens_cc.pdf')

In [128]:
count = len([x for x in p_value_list3 if x < 0.0003937])
count

98

In [129]:
count = len([x for x in correlation_list3 if x > 0.2])
count

118

In [130]:
gene_names = list(gene_data_dict.keys())
# Create the bar chart
fig = go.Figure(go.Bar(
    x=correlation_list3,
    y=gene_names,
    orientation='h',  # Horizontal bar chart
    marker=dict(color=correlation_list1, coloraxis="coloraxis")  # Color based on the correlation factor
))
fig.add_shape(
    type="line",
    x0=0.2, y0=0, x1=0.2, y1=len(gene_names),
    line=dict(color="yellow", width=3, dash="dashdot"),
)


# Update layout for better visualization
fig.update_layout(
    xaxis_title="Correlation Coefficient",
    yaxis_title="Gene ID",
    coloraxis=dict(colorscale='Viridis'),  # Red for higher, blue for lower values
    height=1200  # Adjust height based on the number of genes to avoid squeezing
)

fig.show()
fig.write_image('ingroup_jsd_ens_cc_bar.pdf')

In [93]:
# Function to plot gene data
def plot_gene_data(gene_data_dict):
    keys = list(gene_data_dict.keys())
    rows = int(len(keys) ** 0.5) + 1  # Calculate the number of rows for subplots
    cols = (len(keys) + rows - 1) // rows  # Calculate the number of columns

    fig = make_subplots(rows=rows, cols=cols, subplot_titles=[f'{key}' for key in keys])
    
    # Populate subplots
    for index, key in enumerate(keys, start=1):
        gene_data = gene_data_dict[key]
        jsd_diff = gene_data['jsd_diff']
        ens_abs_diff = gene_data['ens_abs_diff']

        row = (index - 1) // cols + 1
        col = (index - 1) % cols + 1
        
        fig.add_trace(
            go.Scatter(
                x=np.sqrt(jsd_diff),
                y=np.sqrt(ens_abs_diff),
                mode='markers',
                name=f'{key}'
            ),
            row=row,
            col=col
        )
        
        # Adding a trend line
        fig.add_trace(
            go.Scatter(
                x=np.sqrt(jsd_diff),
                y=np.poly1d(np.polyfit(np.sqrt(jsd_diff), np.sqrt(ens_abs_diff), 1))(np.sqrt(jsd_diff)),
                mode='lines',
                name=f'Trend {key}',
                line=dict(color='red')
            ),
            row=row,
            col=col
        )
        
        # Update axis properties
        fig.update_xaxes(title_text="jsd_diff" if row == rows else "", row=row, col=col)
        fig.update_yaxes(title_text="ENS Difference" if col == 1 else "", row=row, col=col)
    
    fig.update_layout(
        height=300 * rows,  # Set a reasonable height based on the number of rows
        width=300 * cols,   # Set a reasonable width based on the number of columns
        title_text="Scatter Plots of Ingroup JSD vs. ENS Difference",
        showlegend=False
    )
    
    return fig

In [94]:
# Usage example
fig = plot_gene_data(gene_data_dict)
fig.show()